In [5]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../../data/'
group_dir_ = os.path.join('../../../HPCF/all/test_0.8/data/groups/random')
folder = 'all (user)'
file_name = 'normalized_to_rating_filter_track_5_user_100.pkl'

In [6]:
predictions = pd.read_pickle(os.path.join(dir_, folder, 'topN_pred_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, folder, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])[:5]

FileNotFoundError: [Errno 2] No such file or directory: '../../../data/all (user)/topN_pred_normalized_log_filter_track_5_user_100.pkl'

In [3]:
all_user = pd.read_pickle(os.path.join('../../../HPCF/all/test_0.8/data/', file_name[:-3] + 'pkl'))
df = pd.read_pickle(os.path.join('../../../HPCF/all/test_0.8/data/', file_name[:-3] + 'pkl'))

In [4]:
predictions[:5]

,uid,tid,rating
0,0,4,0.332034
1,0,14,0.487476
2,0,19,0.393657
3,0,21,0.351009
4,0,27,0.322608


In [5]:
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,rating,count
21,0,4,0.903090,7
31,0,14,1.361728,22
42,0,19,0.778151,5
44,0,21,0.954243,8
59,0,27,0.477121,2


In [6]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [7]:
num_user = len(test['uid'].unique())
num_user

953

In [8]:
pop_dict = {}
for i in all_user['tid'].unique():
    pop_dict[i] = len(all_user[all_user['tid']==i])/num_user

In [9]:
top_n_size = 50
top_n_lists_g = []
top_n_items_g = []
for groups in groups_n:

    group_size = len(groups[0])
    top_n_lists = []
    top_n_items = pd.DataFrame()

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 
        
        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)

        if len(top_n_items) == 0:
            top_n_items = rating_table
        else:
            top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()

    top_n_lists_g.append(top_n_lists)   
    top_n_items_g.append(top_n_items)

COV = []    
NOV = []
HitNOV = []
HitCOV = []
for i in range(len(groups_n)):
    groups = groups_n[i]
    top_n_lists = top_n_lists_g[i]
    top_n_items = top_n_items_g[i]

    nov = 0
    hitNov = 0
    hits = np.zeros(len(df['tid'].unique()), dtype=bool)
    for j in (range(len(groups))):
        group = groups[j]
        top_n_list = top_n_lists[j]

        for k in range(len(group)):
            for l in range(top_n_size):
                uid = group[k]
                tid = top_n_list[l]
                t = test[test['uid'] == uid]
                t = t[t['tid'] == tid]
                nov += (1-pop_dict[tid])
                if len(t) > 0:
                    hits[int(tid)] = True
                    hitNov += (1-pop_dict[tid])
    NOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
    COV.append(len(top_n_items)/len(df['tid'].unique()))
    HitNOV.append(hitNov/top_n_size/(len(groups)*len(groups[0])))
    HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
print(NOV)
print(HitNOV)
print(COV)
print(HitCOV)

[0.6367998007177571, 0.6639315328317301, 0.6768900508787169, 0.6859777986414463, 0.6937181717959814, 0.6954454971915555, 0.7001841156190023, 0.7013347324239606, 0.7032975092505943]
[0.07920335605385889, 0.07531125903809169, 0.06965804579964172, 0.06685359253327428, 0.06354952824967545, 0.06256220956378353, 0.060305492606276365, 0.05936481287163334, 0.05740118186336773]
[0.0019356845024656177, 0.0022022377782149817, 0.0024878305736607286, 0.0029257395266775402, 0.0027924628888028585, 0.0028368884347610857, 0.003020937125159456, 0.002989204592332151, 0.0031097882170759107]
[0.0017516358120672475, 0.0020626146337748385, 0.002272049350435053, 0.0026782257706245596, 0.002570335159011722, 0.0026401467312317933, 0.0028368884347610857, 0.0028368884347610857, 0.0027988093953683194]
